### Connect To Workspace

In [17]:
from azureml.core import Workspace

workspace = Workspace.from_config()

### Get dataset and compute target

In [18]:
pokemon_dataset = workspace.datasets['Pokemon Dataset']
compute_target = workspace.compute_targets['k80cluster']

### Create experiment

In [19]:
from azureml.core import Experiment

experiment_name = 'pokemon-classifier'
experiment = Experiment(workspace, name=experiment_name)

### Submit Job

In [9]:
%pycat training/train.py

In [26]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory='training',
                      entry_script='train.py',
                      compute_target=compute_target,
                      script_params = {
                          '--data_dir': pokemon_dataset.as_named_input('pokemon').as_mount(),
                          '--image_dim': 96,
                          '--learning_rate': 0.001,
                          '--batch_size': 32,
                          '--steps_per_epoch': 25,
                          '--num_epochs': 20,
                          '--output_dir':'./outputs',
                      },
                      framework_version='2.0',
                      use_gpu=True,
                      pip_packages=['Pillow==6.2.0', 'scipy==1.1.0', 'azureml-dataprep[fuse,pandas]==1.1.29'])

In [ ]:
run = experiment.submit(estimator)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

### Register model

In [ ]:
model = run.register_model(model_name='pokemon-classifier', model_path='outputs')